In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Install Required Libraries

In [ ]:
!pip install torchvision
!pip install faiss-gpu
!pip install -q -U transformers datasets bitsandbytes peft evaluate sentence-transformers
!pip install --upgrade transformers
!pip install ipywidgets
!pip install --upgrade ipywidgets

Import Libraries and Configure Environment

In [ ]:
import os
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
from sentence_transformers import SentenceTransformer, util
from accelerate import Accelerator
import faiss
from transformers import pipeline
import evaluate
import shutil

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Clear the cache
cache_dir = os.path.expanduser("~/.cache/huggingface")
shutil.rmtree(cache_dir, ignore_errors=True)

# Clear GPU memory
torch.cuda.empty_cache()

Load Model and Tokenizer

In [ ]:
# Verify the model name
model_name = "/kaggle/input/gemma/transformers/2b-it/3"  # Use the gemma model

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)

print("Model and tokenizer loaded successfully!")

# Inspect the model architecture to find the correct target modules
for name, module in model.named_modules():
    print(name)

Configure LoRA

In [ ]:
# Update the target modules based on the inspection
target_modules = [
    "model.layers.0.self_attn.q_proj",
    "model.layers.0.self_attn.k_proj",
    "model.layers.0.self_attn.v_proj",
    "model.layers.1.self_attn.q_proj",
    "model.layers.1.self_attn.k_proj",
    "model.layers.1.self_attn.v_proj",
    # Add more layers as needed
]

# LoRA Configuration
peft_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM", target_modules=target_modules)
model = get_peft_model(model, peft_config)

Load and Preprocess Dataset

In [ ]:
# Load Dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Preprocess Data
def preprocess_function(examples):
    for i in range(len(examples['Response'])):
        examples['Response'][i] = "<user> " + examples['Response'][i] + " <counselor> "
    return examples

dataset = dataset.map(preprocess_function, batched=True)

# Split dataset
train_test_split = dataset['train'].train_test_split(test_size=0.1, seed=42)
train_data = train_test_split['train']
val_data = train_test_split['test']

# Check for empty strings and filter by token length
val_data = val_data.filter(lambda example: len(tokenizer(example['Response'], truncation=True, max_length=512)['input_ids']) > 1)

Prepare Embeddings with SentenceTransformer and FAISS

In [ ]:
# Initialize SentenceTransformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')  # Experiment with different models

# Prepare Data Embeddings with FAISS in Batches
batch_size = 128  # Adjust as needed
train_embeddings = []
for i in range(0, len(train_data['Response']), batch_size):
    batch = train_data['Response'][i:i + batch_size]
    train_embeddings.extend(sentence_model.encode(batch))
train_embeddings = np.array(train_embeddings)
index = faiss.IndexFlatL2(train_embeddings.shape[1])
index.add(train_embeddings)

Define Chatbot Response and Reward Functions

In [ ]:
# Semantic Search with FAISS and Keyword-Based Search
def semantic_chatbot_response(user_input):
    user_embedding = sentence_model.encode(user_input)
    D, I = index.search(np.expand_dims(user_embedding, axis=0), k=1)  # Top 1 result
    most_similar_response = train_data['Response'][I[0][0]]

    # Keyword-based search
    keyword_responses = [response for response in train_data['Response'] if any(keyword in response for keyword in user_input.split())]
    if keyword_responses:
        most_similar_response = keyword_responses[0]

    if D[0][0] > 0.7:  # Adjust threshold for similarity
        print("(Generating response with language model...)")
        input_ids = tokenizer(user_input, return_tensors="pt").input_ids.to(model.device)
        with torch.no_grad():
            output = model.generate(
                input_ids,
                max_new_tokens=50,
                do_sample=False,
                top_k=50,
                temperature=0.7,
                eos_token_id=tokenizer.eos_token_id
            )
        most_similar_response = tokenizer.decode(output[0], skip_special_tokens=True)

    return most_similar_response

def get_reward(response, user_input):
    """
    Calculates a reward for the chatbot's response based on various criteria.

    Args:
      response: The chatbot's generated response.
      user_input: The user's input to the chatbot.

    Returns:
      A float representing the overall reward for the response.
    """
    # 1. Distinctness from Input
    response_embedding = sentence_model.encode(response)
    input_embedding = sentence_model.encode(user_input)
    similarity = util.cos_sim(response_embedding, input_embedding)[0][0].item()
    distinctness_score = 1.0 - similarity

    # 2. Conciseness
    conciseness_score = 1.0 / len(response.split())

    # 3. Empathy/Helpfulness (using a sentiment analysis pipeline)
    classifier = pipeline("sentiment-analysis") 
    sentiment_result = classifier(response)
    empathy_score = sentiment_result[0]['score']

    # 4. Safety (using a toxicity detection pipeline)
    toxicity_classifier = pipeline("toxicity-detection")
    toxicity_result = toxicity_classifier(response)
    safety_score = 1.0 - toxicity_result[0]['score']

    # Combine scores (adjust weights as needed)
    overall_reward = (
        distinctness_score * 0.3 + 
        conciseness_score * 0.2 + 
        empathy_score * 0.3 + 
        safety_score * 0.2
    )  

    return overall_reward

Train and Run Chatbot

In [ ]:
def rl_update(model, optimizer, user_input_ids, response_ids, reward):
    model.train()
    
    max_length = max(len(ids[0]) for ids in [user_input_ids, response_ids])
    
    padded_user_input_ids = tokenizer.pad({'input_ids': user_input_ids}, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt").input_ids.to(model.device)
    padded_response_ids = tokenizer.pad({'input_ids': response_ids}, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt").input_ids.to(model.device)
    
    outputs = model(padded_user_input_ids, labels=padded_response_ids)
    loss = outputs.loss * reward
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# Chatbot Interaction
def run_chatbot():
    print("Welcome to the Wellness Chatbot! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Take care! I'm here if you need to talk again.")
            break
        # Format the input
        formatted_input = f"<user> {user_input} <counselor>"
        input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids.to(model.device)
        with torch.no_grad():
            output = model.generate(input_ids, max_new_tokens=50, eos_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        # Extract the counselor's response
        counselor_response = response.split("<counselor>")[-1].strip()
        print(f"Chatbot: {counselor_response}")

# Start the chatbot
run_chatbot()

Calculate Perplexity

In [ ]:
def calculate_perplexity_iterative(model, tokenizer, dataset, batch_size=1):
    model.eval()
    total_perplexity = 0
    num_batches = 0
    with torch.no_grad():
        for i in range(0, len(dataset), batch_size):
            batch = dataset[i:i + batch_size]
            inputs = tokenizer(batch['Response'], return_tensors="pt", padding=True, truncation=True, max_length=512)
            inputs = inputs.to(model.device)
            with torch.cuda.amp.autocast():
                outputs = model(**inputs, labels=inputs['input_ids'])
            log_prob = outputs.loss
            perplexity = np.exp(log_prob.item())
            total_perplexity += perplexity
            num_batches += 1
            print(f"Batch {num_batches} perplexity: {perplexity}")
    avg_perplexity = total_perplexity / num_batches
    print(f"Average Perplexity: {avg_perplexity}")

# Test on val data
calculate_perplexity_iterative(model, tokenizer, val_data)